# N46Whisper TTS (CosyVoice)

**文本转语音工具 / Text-to-Speech Tool**

基于阿里通义实验室 CosyVoice 语音合成大模型

---

## 推荐使用方式

由于 CosyVoice 依赖较多，建议使用社区维护的稳定版 Colab：

**👉 [点击打开 CosyVoice Colab](https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/notebooks/CosyVoice.ipynb)**

### 功能特点
- ✅ 一键启动，无需手动安装依赖
- ✅ 支持本地上传音频进行声音克隆
- ✅ 支持中文、英语、日语、韩语、粤语
- ✅ 高质量神经网络语音合成

### 使用教程
- [B站视频教程](https://www.bilibili.com/video/BV1vz421q7ir/)

---

*如果你想在本地运行，可以继续执行以下单元格（需要较长时间安装依赖）*

In [ ]:
#@title **1. 安装依赖 / Install Dependencies**

# @markdown <font size="2">安装 CosyVoice 核心依赖（约 3-5 分钟）</font>

import os

# 克隆 CosyVoice 仓库
if not os.path.exists('/content/CosyVoice'):
    !git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git /content/CosyVoice
    %cd /content/CosyVoice
    !git submodule update --init --recursive

%cd /content/CosyVoice

# 安装核心依赖（精简版，避免安装过多不必要的包）
!pip install -q conformer==0.3.2 diffusers==0.32.2 gdown==5.2.0 gradio==5.23.0
!pip install -q grpcio==1.63.0 grpcio-tools==1.63.0 hydra-core==1.3.2 HyperPyYAML==1.2.2
!pip install -q inflect==7.3.1 lightning==2.4.0 matplotlib==3.9.2 modelscope==1.22.3
!pip install -q networkx==3.3 omegaconf==2.3.0 onnxruntime-gpu==1.19.0
!pip install -q openai-whisper==20240930 protobuf==5.29.2 pydub==0.25.1 pyloudnorm==0.1.1
!pip install -q pytorch-wavelets==1.3.0 rich==13.7.1 soundfile==0.12.1 tensorboard==2.18.0
!pip install -q torch==2.4.0 torchaudio==2.4.0 wget==3.2 WeTextProcessing==1.0.4.1

# 安装系统依赖
!apt-get install -q -y sox libsox-dev > /dev/null 2>&1

from IPython.display import clear_output
clear_output()

print("✅ 依赖安装完成！/ Dependencies installed!")
print("请继续执行下一步：下载模型 / Please continue to next step: Download model")

In [ ]:
#@title **2. 下载模型 / Download Model**

# @markdown <font size="2">从 ModelScope 下载 CosyVoice2 模型（约 1GB，5-10 分钟）</font>

import os

%cd /content/CosyVoice

# 使用 git clone 直接从 ModelScope 下载模型（更稳定）
model_dir = '/content/CosyVoice/pretrained_models/CosyVoice2-0.5B'
if not os.path.exists(model_dir):
    !git clone https://www.modelscope.cn/iic/CosyVoice2-0.5B.git {model_dir}

from IPython.display import clear_output
clear_output()

# 验证模型
if os.path.exists(f"{model_dir}/llm.pt"):
    print("✅ 模型下载完成！/ Model downloaded!")
    print(f"模型路径 / Model path: {model_dir}")
else:
    print("❌ 模型下载可能失败，请重新运行此单元格")
    print("❌ Model download may have failed, please re-run this cell")

In [ ]:
#@title **3. TTS 参数设置 / TTS Settings**

# @markdown **选择语音语言 / Select Voice Language**
tts_language = "中文"  # @param ["中文", "英语", "日语", "韩语", "粤语"]

# @markdown **输出音频格式 / Output Audio Format**
tts_output_format = "wav"  # @param ["wav", "mp3"]

# @markdown **使用指令模式 / Use Instruct Mode**
# @markdown <font size="2">可添加情感、语速、方言等指令</font>
use_instruct = False  # @param {type:"boolean"}

# @markdown **指令内容 / Instruction** <font size="2">(如：用四川话说、用开心的语气说)</font>
instruct_text = "用温柔的语气说"  # @param {type:"string"}

print(f"TTS 配置完成 / TTS Settings configured:")
print(f"  语言/Language: {tts_language}")
print(f"  输出格式/Format: {tts_output_format}")
print(f"  指令模式/Instruct: {use_instruct}")
if use_instruct:
    print(f"  指令/Instruction: {instruct_text}")

In [ ]:
#@title **4. 运行 TTS / Run TTS**

# @markdown <font size="2">上传文本文件并执行文本转语音转换</font>

import os
import re
import sys
from pathlib import Path

%cd /content/CosyVoice
sys.path.insert(0, '.')
sys.path.insert(0, 'third_party/Matcha-TTS')

from google.colab import files
from IPython.display import clear_output, Audio, display

# 上传文本文件
print("请上传文本文件 (.txt, .srt, .ass) / Please upload text file (.txt, .srt, .ass)")
uploaded = files.upload()
tts_input_file = list(uploaded.keys())[0]
tts_basename = Path(tts_input_file).stem

clear_output()
print(f"已加载文件 / File loaded: {tts_input_file}")

# 读取并预处理文本
def read_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    
    ext = Path(filepath).suffix.lower()
    
    if ext == '.srt':
        lines = content.split('\n')
        text_lines = []
        for line in lines:
            line = line.strip()
            if not line or line.isdigit() or '-->' in line:
                continue
            text_lines.append(line)
        return ' '.join(text_lines)
    
    elif ext == '.ass':
        lines = content.split('\n')
        text_lines = []
        for line in lines:
            if line.startswith('Dialogue:'):
                parts = line.split(',', 9)
                if len(parts) >= 10:
                    text = parts[9]
                    text = re.sub(r'\{[^}]*\}', '', text)
                    text = text.replace('\\N', ' ').replace('\\n', ' ')
                    text_lines.append(text.strip())
        return ' '.join(text_lines)
    
    else:
        return content

text_content = read_text_file(tts_input_file)
print(f"文本长度 / Text length: {len(text_content)} 字符/characters")
print(f"文本预览 / Preview: {text_content[:100]}...")

# 加载 CosyVoice2 模型
print("\n加载 CosyVoice2 模型 / Loading CosyVoice2 model...")
from cosyvoice.cli.cosyvoice import CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio
import torch

model_dir = 'pretrained_models/CosyVoice2-0.5B'
cosyvoice = CosyVoice2(model_dir, load_jit=False, load_trt=False)

clear_output()
print("✅ 模型加载完成 / Model loaded!")

# 文本分段处理（CosyVoice 对长文本需要分段）
def split_text(text, max_length=200):
    sentences = re.split(r'([。！？.!?\n])', text)
    segments = []
    current_segment = ""
    
    for i in range(0, len(sentences)-1, 2):
        sentence = sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '')
        if len(current_segment) + len(sentence) <= max_length:
            current_segment += sentence
        else:
            if current_segment.strip():
                segments.append(current_segment.strip())
            current_segment = sentence
    
    if current_segment.strip():
        segments.append(current_segment.strip())
    
    if not segments:
        for i in range(0, len(text), max_length):
            segments.append(text[i:i+max_length])
    
    return segments

text_segments = split_text(text_content)
print(f"分段数量 / Number of segments: {len(text_segments)}")

# 加载参考音频（用于声音克隆）
sample_rate = 24000
ref_audio_path = 'asset/zero_shot_prompt.wav'
ref_audio = load_wav(ref_audio_path, sample_rate)
ref_text = "希望你以后能够做的比我还好呦。"

# 生成音频
print("生成音频中 / Generating audio...")
from tqdm import tqdm

audio_segments = []

for i, segment in enumerate(tqdm(text_segments, desc="TTS Progress")):
    if not segment.strip():
        continue
    
    try:
        if use_instruct:
            # 指令模式
            for result in cosyvoice.inference_instruct2(
                segment, 
                instruct_text,
                ref_audio,
                stream=False
            ):
                audio_segments.append(result['tts_speech'])
                break
        else:
            # Zero-shot 模式
            for result in cosyvoice.inference_zero_shot(
                segment, 
                ref_text,
                ref_audio,
                stream=False
            ):
                audio_segments.append(result['tts_speech'])
                break
    except Exception as e:
        print(f"段落 {i} 生成失败 / Segment {i} failed: {e}")
        continue

print(f"\n生成了 {len(audio_segments)} 个音频段落 / Generated {len(audio_segments)} audio segments")

# 合并音频
print("合并音频文件 / Merging audio files...")
combined_audio = torch.cat(audio_segments, dim=1)

# 保存音频
output_filename = f"{tts_basename}_tts.wav"
torchaudio.save(output_filename, combined_audio, sample_rate)

# 转换格式（如果需要 mp3）
if tts_output_format == "mp3":
    from pydub import AudioSegment
    audio = AudioSegment.from_wav(output_filename)
    mp3_filename = f"{tts_basename}_tts.mp3"
    audio.export(mp3_filename, format="mp3", bitrate="192k")
    os.remove(output_filename)
    output_filename = mp3_filename

print(f"\n✅ 音频生成完成 / Audio generation complete: {output_filename}")

# 显示音频预览
print("\n音频预览 / Audio preview:")
display(Audio(output_filename))

# 触发下载
files.download(output_filename)

print("\n✅ TTS 转换完成！/ TTS conversion complete!")